# Projet commencé le 04/02/2019

Lien utiles :
* [https://medium.com/@smallfishbigsea/faster-r-cnn-explained-864d4fb7e3f8](http://)
* [https://arxiv.org/pdf/1807.07327.pdf](http://) 
* [https://keras.io/getting-started/faq/#how-can-i-obtain-the-output-of-an-intermediate-layer](http://)

## Imports :

In [51]:
# Algèbre linéaire
import numpy as np

# Data processing, CSV
import pandas as pd

# Division de float
from __future__ import division

# Type Image
from PIL import Image

# Image plot
import matplotlib.pyplot as plt

# Récupère l'image d'une requête
from io import BytesIO

# Requête sur URL
import requests

# Permet l'accès à Big Querry
import bq_helper

# Mélange du dataset et séparation train/test
from sklearn.model_selection import train_test_split

# Partie machine learning
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Convolution2D, Flatten, MaxPooling2D, Dropout, Activation
from keras.utils import to_categorical
from keras.models import load_model
from keras.models import Sequential
from keras.layers.normalization import BatchNormalization

# Utilitaire
import os

In [52]:
# INIT
open_images = bq_helper.BigQueryHelper(active_project="bigquery-public-data", dataset_name="open_images")

Using Kaggle's public dataset BigQuery integration.


## Définitions de fonctions :

In [53]:
# Return l'image trouvable sur une URL
def images_from_url(url):
    response = requests.get(url)
    return Image.open(BytesIO(response.content))

In [54]:
# Return le résultat de la querry
def querry_dataset(size):
    print("Loading dataset...")
    subQueryImages = """
    (SELECT image_id, thumbnail_300k_url
    FROM `bigquery-public-data.open_images.images`
    WHERE thumbnail_300k_url IS NOT NULL)
    """

    subQueryBox = """
    (SELECT image_id, label_name, x_min, x_max, y_min, y_max
    FROM `bigquery-public-data.open_images.annotations_bbox`
    ORDER BY image_id
    LIMIT """ + str(size) + """)
    """

    subQueryWord = """
    (SELECT label_name, label_display_name
    FROM `bigquery-public-data.open_images.dict`)
    """

    labelQuery = """
    (SELECT ROW_NUMBER() OVER (ORDER BY wrd.label_name) - 1 AS row, wrd.label_name
    FROM (SELECT DISTINCT(label_name) FROM """ + subQueryBox + """) lab
    JOIN """ + subQueryWord + """ wrd ON lab.label_name = wrd.label_name)"""

    mainQuery = """
    SELECT img.thumbnail_300k_url, wrd.label_display_name, box.label_name, idx.row, box.x_min, box.x_max, box.y_min, box.y_max
    FROM """ + subQueryImages + """ img
    INNER JOIN """ + subQueryBox + """ box ON img.image_id = box.image_id
    INNER JOIN """ + subQueryWord + """ wrd ON box.label_name = wrd.label_name
    INNER JOIN """ + labelQuery + """ idx ON idx.label_name = wrd.label_name
    ORDER BY img.thumbnail_300k_url
    """
    print("Dataset loaded")
    return open_images.query_to_pandas_safe(mainQuery)

In [135]:
# Charge les images et les labels(bbox et mots) dans la ram
def load_data(data_start, data_length, image_size):
    print("Loading images from URL... (From " + str(data_start) + " to " + str(data_start + data_length) + " with a image size of " + str(image_size) + ")")
    
    for data_ex in range (data_start, data_start + data_length):
        # Récupere l'image et la resize
        image = images_from_url(dataset.thumbnail_300k_url[data_ex])
        image = np.array(image.resize((image_size, image_size)))
        
        # Fait un one-hot pour les labels "mots"
        if (len(image.shape) == 3): # Filtre les images qui sont pas suppr
            features[data_ex - data_start] = image/255
            labels_word[data_ex - data_start][dataset.row[data_ex]] = 1
            labels_bbox[data_ex - data_start] = [dataset.x_min[data_ex], dataset.y_min[data_ex], dataset.x_max[data_ex], dataset.y_max[data_ex]]
            labels.append([labels_word[data_ex - data_start], [dataset.x_min[data_ex], dataset.y_min[data_ex], dataset.x_max[data_ex], dataset.y_max[data_ex]]])
            
    lables = np.array(labels)
    
    X_train, X_valid, y_train, y_valid = train_test_split(features, labels, test_size=0.1, random_state=42)

    y_train = np.array(y_train)
    y_valid = np.array(y_valid)
    print(y_train.shape)
    
    y_train_word = [y_train[i][0] for i in range(0, y_train.shape[0])]
    y_train_word = np.array(y_train_word)
    print(y_train_word.shape)
    
    y_train_bbox = y_train[:, 1]
    y_valid_word = y_valid[:, 0]
    y_valid_bbox = y_valid[:, 1]
    
    print("Image loaded")
    
    return [X_train, X_valid, y_train_word, y_train_bbox, y_valid_word, y_valid_bbox]
            

In [136]:
def crop_image(image, image_size, x_min, y_min, x_max, y_max):
    image.resize((image_size, image_size))
    box_rectangle = (image_size*x_min, image_size*y_min, image_size*x_max, image_size*y_max)
    image = image.crop(box_rectangle)
    image = np.array(image)

In [137]:
def create_anchor(x, y, w, h):
    return [x-(w/2), y-(h/2), x+(w/2), y+(h/2)]

In [138]:
dataset_size = 100
sub_dataset_size = 100
image_size = 224

features = np.ndarray(shape=(sub_dataset_size, image_size, image_size, 3), dtype=float)
labels_word = np.zeros(shape=(sub_dataset_size, 600), dtype=float)
labels_bbox = np.ndarray(shape=(sub_dataset_size, 4), dtype=float)
labels = []

dataset = querry_dataset(dataset_size)

Loading dataset...
Dataset loaded


In [139]:
X_train, X_valid, y_train_word, y_train_bbox, y_valid_word, y_valid_bbox = load_data(0, 100, 224)


"""X_train = data[0]
X_valid = data[1]
y_train_word = data[2]
y_train_bbox = data[3]
y_valid_word = data[4]
y_valid_bbox = data[5]"""

Loading images from URL... (From 0 to 100 with a image size of 224)
(90, 2)
(90, 600)
Image loaded


'X_train = data[0]\nX_valid = data[1]\ny_train_word = data[2]\ny_train_bbox = data[3]\ny_valid_word = data[4]\ny_valid_bbox = data[5]'

In [143]:
#Instantiate an empty model
model = Sequential()

# 1st Convolutional Layer
model.add(Conv2D(filters=96, input_shape=(224,224,3), kernel_size=(11,11), strides=(4,4), padding='valid'))
model.add(Activation('relu'))
# Max Pooling
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='valid'))

# 2nd Convolutional Layer
model.add(Conv2D(filters=256, kernel_size=(11,11), strides=(1,1), padding='valid'))
model.add(Activation('relu'))
# Max Pooling
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='valid'))

# 3rd Convolutional Layer
model.add(Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), padding='valid'))
model.add(Activation('relu'))

# 4th Convolutional Layer
model.add(Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), padding='valid'))
model.add(Activation('relu'))

# 5th Convolutional Layer
model.add(Conv2D(filters=256, kernel_size=(3,3), strides=(1,1), padding='valid'))
model.add(Activation('relu'))
# Max Pooling
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='valid'))

# Passing it to a Fully Connected layer
model.add(Flatten())
# 1st Fully Connected Layer
model.add(Dense(4096, input_shape=(224*224*3,)))
model.add(Activation('relu'))
# Add Dropout to prevent overfitting
model.add(Dropout(0.4))

# 2nd Fully Connected Layer
model.add(Dense(4096))
model.add(Activation('relu'))
# Add Dropout
model.add(Dropout(0.4))

# 3rd Fully Connected Layer
model.add(Dense(1000))
model.add(Activation('relu'))
# Add Dropout
model.add(Dropout(0.4))

# Output Layer
model.add(Dense(600))
model.add(Activation('softmax'))

model.summary()

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=["accuracy"])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_86 (Conv2D)           (None, 54, 54, 96)        34944     
_________________________________________________________________
activation_154 (Activation)  (None, 54, 54, 96)        0         
_________________________________________________________________
max_pooling2d_52 (MaxPooling (None, 27, 27, 96)        0         
_________________________________________________________________
conv2d_87 (Conv2D)           (None, 17, 17, 256)       2973952   
_________________________________________________________________
activation_155 (Activation)  (None, 17, 17, 256)       0         
_________________________________________________________________
max_pooling2d_53 (MaxPooling (None, 8, 8, 256)         0         
_________________________________________________________________
conv2d_88 (Conv2D)           (None, 6, 6, 384)         885120    
__________

In [145]:
model.fit(X_train, y_train_word, nb_epoch=5, batch_size=1)

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:1: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  """Entry point for launching an IPython kernel.


Epoch 1/5
90/90 [==============================] - 5s 52ms/step - loss: 3.8174 - acc: 0.0889
Epoch 2/5
90/90 [==============================] - 4s 49ms/step - loss: 3.7835 - acc: 0.1000
Epoch 3/5
90/90 [==============================] - 4s 49ms/step - loss: 3.5568 - acc: 0.1333
Epoch 4/5
90/90 [==============================] - 4s 49ms/step - loss: 3.5465 - acc: 0.0889
Epoch 5/5
90/90 [==============================] - 4s 49ms/step - loss: 3.4970 - acc: 0.1222


In [ ]:
# Image -> Convolution
# Convolution + Chaques anchor -> Croped Image
# Croped Image -> RPN (Objet ? Oui/Non + Vrai Coo) (Pour backprop, check toutes les BBOXs de l'image puis IoU)
# Si object, on crop sur les vrais coos -> Classifier

model = Sequential()

model.add(Conv2D(96, kernel_size=(11, 11), strides=4, activation='relu',input_shape=(image_size_x,image_size_y,3)), name='input conv')
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(256, kernel_size=(5, 5),activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(384, kernel_size=(3, 3),activation='relu'))
model.add(Conv2D(384, kernel_size=(3, 3),activation='relu'))
model.add(Conv2D(256, kernel_size=(3, 3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2), name='output conv'))

#https://keras.io/getting-started/faq/#how-can-i-obtain-the-output-of-an-intermediate-layer

model.add(Flatten())
model.add(Dense(4096, activation='relu'))
model.add(Dense(4096, activation='relu'))
model.add(Dense(600, activation='softmax'))

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
#model.save('my_model.h5')

for sub_dataset_ex in range (0, int(dataset_size/sub_dataset_size)):
    load_data(sub_dataset_ex*sub_dataset_size, sub_dataset_size, image_size)
    X_train, X_valid, y_train, y_valid = train_test_split(features_croped, labels_croped, test_size=0.1, random_state=42)
    model.fit(X_train, y_train, validation_data=(X_valid, y_valid), epochs=100, batch_size=4)
    #model.save('my_model.h5')


In [ ]:
#model.fit(X_train, y_train, validation_data=(X_valid, y_valid), epochs=100, batch_size=128)

#print(y_train[1])

for i in range (0, sub_dataset_size):
    plt.imshow(X_train[i])a
    plt.show()
    
"""# Récupere la sous image
        width, height = image.size
        box_rectangle = (width*dataset.x_min[data_ex], height*dataset.y_min[data_ex], width*dataset.x_max[data_ex], height*dataset.y_max[data_ex])
        image = image.crop(box_rectangle)
        image = np.array(image.resize((image_size_x, image_size_y)))"""